### Extractive Approach Text Rank Algorithm

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Arshit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv('tennis_articles_v4.csv')

In [3]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [4]:
df.shape

(8, 3)

In [5]:
li_text_to_sentences = []
for i in range(df.shape[0]):
    li = []
    li = df.loc[i][1].split('.')
    li_text_to_sentences.append(li)

In [6]:
lower_li_text_to_sentences = []
for i in range(len(li_text_to_sentences)):
    li=[]
    for j in range(len(li_text_to_sentences[i])):
        lower_sentence = li_text_to_sentences[i][j].lower()
        li.append(lower_sentence)
    lower_li_text_to_sentences.append(li)

In [7]:
li_all_sentences = []
for i in range(len(li_text_to_sentences)):
    for j in range(len(li_text_to_sentences[i])):
        lower_sentence = li_text_to_sentences[i][j].lower()
        print(lower_sentence)
        print()
        if lower_sentence == '' or lower_sentence == "'" or lower_sentence == '"':
            pass
        else:
            li_all_sentences.append(lower_sentence)

maria sharapova has basically no friends as tennis players on the wta tour

 the russian player has no problems in openly speaking about it and in a recent interview she said: 'i don't really hide any feelings too much

 i think everyone knows this is my job here

 when i'm on the courts or when i'm on the court playing, i'm a competitor and i want to beat every single person whether they're in the locker room or across the net

so i'm not the one to strike up a conversation about the weather and know that in the next few minutes i have to go and try to win a tennis match

 i'm a pretty competitive girl

 i say my hellos, but i'm not sending any players flowers as well

 uhm, i'm not really friendly or close to many players

 i have not a lot of friends away from the courts

' when she said she is not really close to a lot of players, is that something strategic that she is doing? is it different on the men's tour than the women's tour? 'no, not at all

 i think just because you're in 

In [8]:
def read_glove_vecs(glove_file):
    with open(glove_file, encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [9]:
words,word_to_vec_map = read_glove_vecs('glove.6B.200d.txt')

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arshit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [12]:
len(stop_words)

179

In [13]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [14]:
li_all_sentences = [remove_stopwords(r.split()) for r in li_all_sentences]

In [15]:
li_all_sentences = pd.Series(li_all_sentences).str.replace("[^a-zA-Z]", " ")

In [16]:
avg_vectors = []
for j in range(0,len(li_all_sentences)):
    sen_to_words = li_all_sentences[j].split(' ')
    avg=[]
    total=0
    for z in range(len(sen_to_words)):
        try:
            sen_to_words.remove('')
            sen_to_words.remove(' ')
        except:
            pass
    for w in sen_to_words:
        if w in word_to_vec_map.keys():
            total +=word_to_vec_map[w]
        else:
            total += np.zeros((200,))
    for i in range(len(total)):
        avg.append(total[i]/len(sen_to_words))
    avg=np.asarray(avg)
    avg_vectors.append(avg)

In [17]:
len(avg_vectors)

118

In [18]:
len(li_all_sentences)

118

In [19]:
sim_mat = np.zeros([len(li_all_sentences), len(li_all_sentences)])

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
for i in range(len(li_all_sentences)):
    for j in range(len(li_all_sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(avg_vectors[i].reshape(1,200), avg_vectors[j].reshape(1,200))[0,0]

In [22]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [23]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(li_all_sentences)), reverse=True)

In [24]:
len(ranked_sentences)

118

In [25]:
# Text Summary
for i in range(10):
    print(ranked_sentences[i][1])

i m one strike conversation weather know next minutes go try win tennis match
major players feel big event late november combined one january australian open mean much tennis little rest
 i felt like best weeks get know players playing fed cup weeks olympic weeks  necessarily tournaments
speaking swiss indoors tournament play sundays final romanian qualifier marius copil  world number three said given impossibly short time frame make decision  opted commitment
spaniard broke anderson twice second get another chance south african s serve final set
currently ninth place  nishikori win could move within     points cut eight man event london next month
used first break point close first set going     second wrapping win first match point
competition set feature    countries november       finals madrid next year  replace classic home and away ties played four times per year decades
 on monday  free look feel   federer said winning swiss indoors
  said really close lot players  something st

### Abstractive Approach (Amazon Food Reviews Dataset)

In [1]:
import numpy as np
import pandas as pd
import re
import nltk

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [4]:
len(stop_words)

179

In [5]:
df = pd.read_csv('Reviews.csv',nrows=100000)

In [6]:
df.shape

(100000, 10)

In [7]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
df['Text'][0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [9]:
df.drop_duplicates(subset=['Text'],inplace=True)
df.dropna(axis=0,inplace=True) 

In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
df.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [12]:
data=df.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time'],axis=1)

In [13]:
data.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [14]:
print(data.shape)
print(data.isnull().sum())

(88421, 2)
Summary    0
Text       0
dtype: int64


In [15]:
data['Summary'][28]

'YUMMY!'

In [16]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [17]:
len(contractions)

120

In [18]:
texts = []
for i in range(len(data['Text'])):
    words = data['Text'][i].lower().split(' ')
    new_li = []
    for w in words:
        if w in contractions:
            new_li.append(contractions[w])
        else:
            new_li.append(w)
    
    for ele in new_li:
        if ele in stop_words:
            new_li.remove(ele)
            
    string = " ".join(new_li)
    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', string, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub('[m]{2,}', 'mm', text)
    
    texts.append(text)

In [19]:
texts[:5]

['have bought several vitality canned dog food products have found all be good quality  the product looks like stew a processed meat it smells better  labrador finicky she appreciates product better  most ',
 'product arrived labeled jumbo salted peanuts   the peanuts actually small sized unsalted  sure this an error if vendor intended represent product  jumbo  ',
 'confection has around few centuries   light  pillowy citrus gelatin nuts   case filberts  it is cut tiny squares then liberally coated powdered sugar   it is a tiny mouthful heaven   too chewy  very flavorful   highly recommend yummy treat   you familiar story c s  lewis   the lion  witch  the wardrobe    this is the treat seduces edmund selling his brother sisters the witch ',
 'you looking secret ingredient robitussin believe have found it   got in addition the root beer extract ordered  which good  made cherry soda   the flavor very medicinal ',
 'great taffy great price   wide assortment yummy taffy   delivery was very 

In [20]:
summaries = []
for i in range(len(data['Summary'])):
    words = data['Summary'][i].lower().split(' ')
    new_li = []
    for w in words:
        if w in contractions:
            new_li.append(contractions[w])
        else:
            new_li.append(w)
    
    string = " ".join(new_li)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', string, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub('[m]{2,}', 'mm', text)
    
    summaries.append(text)

In [21]:
summaries[:5]

['good quality dog food',
 'not as advertised',
 ' delight  says it all',
 'cough medicine',
 'great taffy']

In [22]:
len(texts),len(summaries)

(88421, 88421)

In [23]:
for i in range(5):
    print("Text : ",texts[i])
    print("Summary : ",summaries[i])
    print()

Text :  have bought several vitality canned dog food products have found all be good quality  the product looks like stew a processed meat it smells better  labrador finicky she appreciates product better  most 
Summary :  good quality dog food

Text :  product arrived labeled jumbo salted peanuts   the peanuts actually small sized unsalted  sure this an error if vendor intended represent product  jumbo  
Summary :  not as advertised

Text :  confection has around few centuries   light  pillowy citrus gelatin nuts   case filberts  it is cut tiny squares then liberally coated powdered sugar   it is a tiny mouthful heaven   too chewy  very flavorful   highly recommend yummy treat   you familiar story c s  lewis   the lion  witch  the wardrobe    this is the treat seduces edmund selling his brother sisters the witch 
Summary :   delight  says it all

Text :  you looking secret ingredient robitussin believe have found it   got in addition the root beer extract ordered  which good  made che

In [24]:
max_len_summary=8
max_len_text = 40

In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
short_texts = []
short_summaries = []
for i in range(len(texts)):
    if len(texts[i].split())<=40 and len(summaries[i].split())<=8:
        short_texts.append(texts[i])
        short_summaries.append('start '+summaries[i]+' end')

In [27]:
short_texts[:5]

['have bought several vitality canned dog food products have found all be good quality  the product looks like stew a processed meat it smells better  labrador finicky she appreciates product better  most ',
 'product arrived labeled jumbo salted peanuts   the peanuts actually small sized unsalted  sure this an error if vendor intended represent product  jumbo  ',
 'you looking secret ingredient robitussin believe have found it   got in addition the root beer extract ordered  which good  made cherry soda   the flavor very medicinal ',
 'great taffy great price   wide assortment yummy taffy   delivery was very quick   your taffy lover  is a deal ',
 'saltwater taffy great flavors soft chewy   candy was individually wrapped well   none the candies stuck together  did happen the expensive version  fralinger s   would highly recommend candy   served at beach themed party everyone loved it ']

In [28]:
short_summaries[:5]

['start good quality dog food end',
 'start not as advertised end',
 'start cough medicine end',
 'start great taffy end',
 'start great   just as good as the expensive brands  end']

In [29]:
len(short_texts),len(short_summaries)

(43499, 43499)

In [30]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(short_texts,short_summaries,test_size=0.1,random_state=0,shuffle=True) 

In [31]:
len(x_tr),len(y_tr)

(39149, 39149)

In [84]:
# Text Tokenizer

tokenizer_text = Tokenizer()
tokenizer_text.fit_on_texts(short_texts)

thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in tokenizer_text.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value

common = tot_cnt-cnt

tokenizer_text = Tokenizer(num_words = common)
tokenizer_text.fit_on_texts(short_texts)
word_index_text = tokenizer_text.word_index
sequences_texts = tokenizer_text.texts_to_sequences(short_texts)
x_tr = pad_sequences(sequences_texts, maxlen=max_len_text,padding='post')

In [85]:
# Summary Tokenizer

tokenizer_summary = Tokenizer()
tokenizer_summary.fit_on_texts(short_summaries)

thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in tokenizer_summary.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value

common_summ = tot_cnt-cnt  

tokenizer_summary = Tokenizer(num_words = common_summ)
tokenizer_summary.fit_on_texts(short_summaries)
word_index_summary = tokenizer_summary.word_index
sequences_summaries = tokenizer_summary.texts_to_sequences(short_summaries)
y_tr = pad_sequences(sequences_summaries, maxlen=max_len_summary,padding='post')

In [34]:
tokenizer_summary.word_counts['start']

43522

In [35]:
len_word_index_text = common+1
len_word_index_summary = common_summ+1

In [36]:
len_word_index_text , len_word_index_summary

(8675, 2044)

In [37]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import random
import matplotlib.pyplot as plt
%matplotlib inline


Using TensorFlow backend.


In [38]:
from attention import AttentionLayer

In [39]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [43]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [44]:
len(x_tr),len(y_tr)

(42374, 42374)

In [45]:

from keras import backend as K 
K.clear_session()

latent_dim = 500
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_len_text,))
enc_emb =  Embedding(len_word_index_text, embedding_dim,trainable=True)(encoder_inputs)

encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Decoder
decoder_inputs = Input(shape=(None,))

dec_emb_layer = Embedding(len_word_index_summary, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

decoder_dense =  TimeDistributed(Dense(len_word_index_summary, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 40, 100)      867500      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 40, 500), (N 1202000     embedding[0][0]                  
_____________________________________________________________________________________________

In [46]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [58]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=30,batch_size=128) 

Train on 42374 samples
Epoch 1/30


25984/42374 [=================>............] - ETA: 24:53 - loss: 3.1551 - acc: 0.49 - ETA: 25:17 - loss: 3.1857 - acc: 0.50 - ETA: 24:55 - loss: 3.2319 - acc: 0.49 - ETA: 24:38 - loss: 3.2412 - acc: 0.50 - ETA: 24:26 - loss: 3.2405 - acc: 0.49 - ETA: 24:20 - loss: 3.2606 - acc: 0.49 - ETA: 24:10 - loss: 3.2245 - acc: 0.50 - ETA: 24:03 - loss: 3.1828 - acc: 0.51 - ETA: 24:06 - loss: 3.1607 - acc: 0.51 - ETA: 24:17 - loss: 3.1694 - acc: 0.51 - ETA: 24:09 - loss: 3.1497 - acc: 0.51 - ETA: 24:01 - loss: 3.1403 - acc: 0.52 - ETA: 24:02 - loss: 3.1374 - acc: 0.51 - ETA: 23:58 - loss: 3.1346 - acc: 0.52 - ETA: 23:54 - loss: 3.1303 - acc: 0.52 - ETA: 23:53 - loss: 3.1243 - acc: 0.52 - ETA: 23:50 - loss: 3.1064 - acc: 0.52 - ETA: 23:46 - loss: 3.0944 - acc: 0.53 - ETA: 23:44 - loss: 3.1072 - acc: 0.52 - ETA: 23:41 - loss: 3.1019 - acc: 0.53 - ETA: 23:37 - loss: 3.0930 - acc: 0.53 - ETA: 23:34 - loss: 3.0867 - acc: 0.53 - ETA: 23:31 - loss: 3.0825 - acc: 0.53 - ETA: 23:29 - loss: 3.0761 - acc: 

42374/42374 [==============================] - ETA: 11:39 - loss: 2.8145 - acc: 0.57 - ETA: 11:34 - loss: 2.8138 - acc: 0.57 - ETA: 11:29 - loss: 2.8137 - acc: 0.57 - ETA: 11:24 - loss: 2.8134 - acc: 0.57 - ETA: 11:18 - loss: 2.8140 - acc: 0.57 - ETA: 11:13 - loss: 2.8135 - acc: 0.57 - ETA: 11:08 - loss: 2.8137 - acc: 0.57 - ETA: 11:02 - loss: 2.8132 - acc: 0.57 - ETA: 10:57 - loss: 2.8133 - acc: 0.57 - ETA: 10:52 - loss: 2.8135 - acc: 0.57 - ETA: 10:47 - loss: 2.8126 - acc: 0.57 - ETA: 10:41 - loss: 2.8128 - acc: 0.57 - ETA: 10:36 - loss: 2.8128 - acc: 0.57 - ETA: 10:30 - loss: 2.8122 - acc: 0.57 - ETA: 10:25 - loss: 2.8116 - acc: 0.57 - ETA: 10:20 - loss: 2.8100 - acc: 0.57 - ETA: 10:14 - loss: 2.8092 - acc: 0.57 - ETA: 10:09 - loss: 2.8072 - acc: 0.57 - ETA: 10:04 - loss: 2.8071 - acc: 0.57 - ETA: 9:58 - loss: 2.8067 - acc: 0.5776 - ETA: 9:53 - loss: 2.8055 - acc: 0.577 - ETA: 9:48 - loss: 2.8054 - acc: 0.577 - ETA: 9:42 - loss: 2.8044 - acc: 0.577 - ETA: 9:37 - loss: 2.8040 - acc: 

26112/42374 [=================>............] - ETA: 38:41 - loss: 2.5915 - acc: 0.58 - ETA: 37:15 - loss: 2.5881 - acc: 0.59 - ETA: 35:56 - loss: 2.6354 - acc: 0.58 - ETA: 35:13 - loss: 2.6184 - acc: 0.59 - ETA: 34:31 - loss: 2.5693 - acc: 0.59 - ETA: 34:12 - loss: 2.6037 - acc: 0.59 - ETA: 33:48 - loss: 2.6003 - acc: 0.59 - ETA: 33:30 - loss: 2.6007 - acc: 0.59 - ETA: 33:11 - loss: 2.5842 - acc: 0.59 - ETA: 32:57 - loss: 2.5924 - acc: 0.59 - ETA: 32:43 - loss: 2.5684 - acc: 0.59 - ETA: 32:31 - loss: 2.5720 - acc: 0.59 - ETA: 32:20 - loss: 2.5924 - acc: 0.59 - ETA: 32:11 - loss: 2.5969 - acc: 0.59 - ETA: 32:09 - loss: 2.5983 - acc: 0.59 - ETA: 31:59 - loss: 2.5890 - acc: 0.59 - ETA: 31:52 - loss: 2.5929 - acc: 0.59 - ETA: 31:43 - loss: 2.5828 - acc: 0.59 - ETA: 31:35 - loss: 2.5874 - acc: 0.59 - ETA: 31:29 - loss: 2.5869 - acc: 0.59 - ETA: 31:22 - loss: 2.5866 - acc: 0.59 - ETA: 31:17 - loss: 2.5818 - acc: 0.59 - ETA: 31:09 - loss: 2.5829 - acc: 0.59 - ETA: 31:04 - loss: 2.5766 - acc: 

42374/42374 [==============================] - ETA: 12:11 - loss: 2.5504 - acc: 0.59 - ETA: 12:06 - loss: 2.5507 - acc: 0.59 - ETA: 12:00 - loss: 2.5500 - acc: 0.59 - ETA: 11:54 - loss: 2.5492 - acc: 0.59 - ETA: 11:48 - loss: 2.5487 - acc: 0.59 - ETA: 11:43 - loss: 2.5480 - acc: 0.59 - ETA: 11:37 - loss: 2.5475 - acc: 0.59 - ETA: 11:31 - loss: 2.5475 - acc: 0.59 - ETA: 11:26 - loss: 2.5476 - acc: 0.59 - ETA: 11:20 - loss: 2.5477 - acc: 0.59 - ETA: 11:14 - loss: 2.5476 - acc: 0.59 - ETA: 11:08 - loss: 2.5474 - acc: 0.59 - ETA: 11:02 - loss: 2.5470 - acc: 0.59 - ETA: 10:57 - loss: 2.5468 - acc: 0.59 - ETA: 10:51 - loss: 2.5471 - acc: 0.59 - ETA: 10:46 - loss: 2.5468 - acc: 0.59 - ETA: 10:40 - loss: 2.5466 - acc: 0.59 - ETA: 10:35 - loss: 2.5457 - acc: 0.59 - ETA: 10:29 - loss: 2.5463 - acc: 0.59 - ETA: 10:23 - loss: 2.5458 - acc: 0.59 - ETA: 10:18 - loss: 2.5448 - acc: 0.59 - ETA: 10:12 - loss: 2.5444 - acc: 0.59 - ETA: 10:06 - loss: 2.5439 - acc: 0.59 - ETA: 10:00 - loss: 2.5439 - acc: 

...

42374/42374 [==============================] - ETA: 12:00 - loss: 1.4889 - acc: 0.69 - ETA: 11:55 - loss: 1.4889 - acc: 0.69 - ETA: 11:51 - loss: 1.4897 - acc: 0.69 - ETA: 11:45 - loss: 1.4897 - acc: 0.69 - ETA: 11:39 - loss: 1.4906 - acc: 0.69 - ETA: 11:34 - loss: 1.4915 - acc: 0.69 - ETA: 11:28 - loss: 1.4919 - acc: 0.69 - ETA: 11:22 - loss: 1.4923 - acc: 0.69 - ETA: 11:16 - loss: 1.4927 - acc: 0.69 - ETA: 11:10 - loss: 1.4934 - acc: 0.69 - ETA: 11:04 - loss: 1.4930 - acc: 0.69 - ETA: 10:58 - loss: 1.4931 - acc: 0.69 - ETA: 10:52 - loss: 1.4932 - acc: 0.69 - ETA: 10:46 - loss: 1.4932 - acc: 0.69 - ETA: 10:40 - loss: 1.4938 - acc: 0.69 - ETA: 10:34 - loss: 1.4929 - acc: 0.69 - ETA: 10:28 - loss: 1.4935 - acc: 0.69 - ETA: 10:22 - loss: 1.4936 - acc: 0.69 - ETA: 10:16 - loss: 1.4940 - acc: 0.69 - ETA: 10:11 - loss: 1.4947 - acc: 0.69 - ETA: 10:05 - loss: 1.4946 - acc: 0.69 - ETA: 9:59 - loss: 1.4948 - acc: 0.6959 - ETA: 9:53 - loss: 1.4947 - acc: 0.695 - ETA: 9:47 - loss: 1.4953 - acc: 

26112/42374 [=================>............] - ETA: 31:14 - loss: 1.4019 - acc: 0.70 - ETA: 31:19 - loss: 1.4217 - acc: 0.70 - ETA: 31:18 - loss: 1.4156 - acc: 0.71 - ETA: 31:13 - loss: 1.4144 - acc: 0.70 - ETA: 31:18 - loss: 1.4257 - acc: 0.70 - ETA: 31:06 - loss: 1.4349 - acc: 0.70 - ETA: 30:57 - loss: 1.4360 - acc: 0.70 - ETA: 30:54 - loss: 1.4398 - acc: 0.70 - ETA: 30:45 - loss: 1.4372 - acc: 0.70 - ETA: 30:38 - loss: 1.4327 - acc: 0.70 - ETA: 30:34 - loss: 1.4354 - acc: 0.70 - ETA: 30:38 - loss: 1.4310 - acc: 0.70 - ETA: 30:35 - loss: 1.4323 - acc: 0.70 - ETA: 30:29 - loss: 1.4422 - acc: 0.70 - ETA: 30:21 - loss: 1.4456 - acc: 0.70 - ETA: 30:16 - loss: 1.4419 - acc: 0.70 - ETA: 30:08 - loss: 1.4442 - acc: 0.70 - ETA: 30:01 - loss: 1.4445 - acc: 0.70 - ETA: 29:57 - loss: 1.4368 - acc: 0.70 - ETA: 29:59 - loss: 1.4405 - acc: 0.70 - ETA: 30:15 - loss: 1.4431 - acc: 0.70 - ETA: 30:14 - loss: 1.4429 - acc: 0.70 - ETA: 30:06 - loss: 1.4408 - acc: 0.70 - ETA: 30:00 - loss: 1.4394 - acc: 

42374/42374 [==============================] - ETA: 12:38 - loss: 1.4712 - acc: 0.69 - ETA: 12:32 - loss: 1.4715 - acc: 0.69 - ETA: 12:26 - loss: 1.4716 - acc: 0.69 - ETA: 12:21 - loss: 1.4718 - acc: 0.69 - ETA: 12:16 - loss: 1.4721 - acc: 0.69 - ETA: 12:11 - loss: 1.4723 - acc: 0.69 - ETA: 12:05 - loss: 1.4722 - acc: 0.69 - ETA: 11:59 - loss: 1.4727 - acc: 0.69 - ETA: 11:53 - loss: 1.4729 - acc: 0.69 - ETA: 11:47 - loss: 1.4737 - acc: 0.69 - ETA: 11:41 - loss: 1.4734 - acc: 0.69 - ETA: 11:35 - loss: 1.4735 - acc: 0.69 - ETA: 11:28 - loss: 1.4736 - acc: 0.69 - ETA: 11:22 - loss: 1.4742 - acc: 0.69 - ETA: 11:16 - loss: 1.4739 - acc: 0.69 - ETA: 11:10 - loss: 1.4738 - acc: 0.69 - ETA: 11:04 - loss: 1.4745 - acc: 0.69 - ETA: 10:58 - loss: 1.4753 - acc: 0.69 - ETA: 10:52 - loss: 1.4755 - acc: 0.69 - ETA: 10:46 - loss: 1.4755 - acc: 0.69 - ETA: 10:40 - loss: 1.4755 - acc: 0.69 - ETA: 10:34 - loss: 1.4757 - acc: 0.69 - ETA: 10:28 - loss: 1.4762 - acc: 0.69 - ETA: 10:21 - loss: 1.4766 - acc: 

26112/42374 [=================>............] - ETA: 32:29 - loss: 1.5438 - acc: 0.68 - ETA: 32:16 - loss: 1.4393 - acc: 0.69 - ETA: 32:07 - loss: 1.4282 - acc: 0.69 - ETA: 32:25 - loss: 1.4326 - acc: 0.69 - ETA: 33:46 - loss: 1.4205 - acc: 0.70 - ETA: 36:05 - loss: 1.4124 - acc: 0.70 - ETA: 35:22 - loss: 1.4123 - acc: 0.70 - ETA: 34:50 - loss: 1.4022 - acc: 0.70 - ETA: 34:28 - loss: 1.4088 - acc: 0.70 - ETA: 34:07 - loss: 1.4080 - acc: 0.70 - ETA: 33:49 - loss: 1.4239 - acc: 0.70 - ETA: 33:29 - loss: 1.4217 - acc: 0.70 - ETA: 33:09 - loss: 1.4207 - acc: 0.70 - ETA: 32:55 - loss: 1.4126 - acc: 0.70 - ETA: 32:41 - loss: 1.4137 - acc: 0.70 - ETA: 32:28 - loss: 1.4107 - acc: 0.70 - ETA: 32:16 - loss: 1.4077 - acc: 0.70 - ETA: 32:04 - loss: 1.4084 - acc: 0.70 - ETA: 31:56 - loss: 1.4016 - acc: 0.70 - ETA: 31:45 - loss: 1.3992 - acc: 0.70 - ETA: 31:32 - loss: 1.3993 - acc: 0.70 - ETA: 31:26 - loss: 1.4045 - acc: 0.70 - ETA: 31:16 - loss: 1.3998 - acc: 0.70 - ETA: 31:08 - loss: 1.3993 - acc: 

42374/42374 [==============================] - ETA: 12:26 - loss: 1.4570 - acc: 0.70 - ETA: 12:20 - loss: 1.4573 - acc: 0.70 - ETA: 12:14 - loss: 1.4572 - acc: 0.70 - ETA: 12:08 - loss: 1.4576 - acc: 0.70 - ETA: 12:03 - loss: 1.4580 - acc: 0.70 - ETA: 11:57 - loss: 1.4584 - acc: 0.70 - ETA: 11:51 - loss: 1.4583 - acc: 0.70 - ETA: 11:46 - loss: 1.4585 - acc: 0.70 - ETA: 11:40 - loss: 1.4587 - acc: 0.70 - ETA: 11:34 - loss: 1.4592 - acc: 0.70 - ETA: 11:28 - loss: 1.4593 - acc: 0.70 - ETA: 11:22 - loss: 1.4593 - acc: 0.70 - ETA: 11:16 - loss: 1.4595 - acc: 0.70 - ETA: 11:10 - loss: 1.4596 - acc: 0.70 - ETA: 11:04 - loss: 1.4601 - acc: 0.70 - ETA: 10:58 - loss: 1.4598 - acc: 0.70 - ETA: 10:52 - loss: 1.4599 - acc: 0.70 - ETA: 10:46 - loss: 1.4600 - acc: 0.70 - ETA: 10:40 - loss: 1.4604 - acc: 0.70 - ETA: 10:34 - loss: 1.4608 - acc: 0.70 - ETA: 10:28 - loss: 1.4607 - acc: 0.70 - ETA: 10:22 - loss: 1.4613 - acc: 0.70 - ETA: 10:16 - loss: 1.4610 - acc: 0.70 - ETA: 10:10 - loss: 1.4612 - acc: 

In [59]:
# encoder inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# decoder inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

decoder_outputs2 = decoder_dense(decoder_inf_concat)

decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [61]:
reverse_target_word_index=tokenizer_summary.index_word  
target_word_index=tokenizer_summary.word_index

In [81]:
def make_predictions(input_seq):
    
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    decoded_seq = np.zeros((1,1))
    decoded_seq[0,0]=target_word_index['start']
    decoded_seq_word = 'start'
    
    
    li=[]
    len_pred_summary = max_len_summary - 1
    
    while decoded_seq_word !='end' and len_pred_summary>=0:
        
        decoded_seq, h,c = decoder_model.predict([decoded_seq] + [e_out,e_h,e_c])
        decoded_seq = np.argmax(decoded_seq[0,-1,:])
        decoded_seq_word = reverse_target_word_index[decoded_seq]
        decoded_seq = np.zeros((1,1))
        decoded_seq[0,0] = target_word_index[decoded_seq_word]
        
        e_h = h
        e_c = c
        
        li.append(decoded_seq_word)
        
        len_pred_summary = len_pred_summary-1
    
    li.remove('end')
    out_final = " ".join(li)
    
    return out_final

In [83]:
for i in range(10):
    print("Text : ",texts[i])
    print("Actual Summary : ",summaries[i])
    print("Predicted Summary : ",make_predictions(x_tr[i].reshape(1,max_len_text)))
    print()

Text :  have bought several vitality canned dog food products have found all be good quality  the product looks like stew a processed meat it smells better  labrador finicky she appreciates product better  most 
Actual Summary :  good quality dog food
Predicted Summary :  good quality dog food

Text :  product arrived labeled jumbo salted peanuts   the peanuts actually small sized unsalted  sure this an error if vendor intended represent product  jumbo  
Actual Summary :  not as advertised
Predicted Summary :  not what i expected

Text :  confection has around few centuries   light  pillowy citrus gelatin nuts   case filberts  it is cut tiny squares then liberally coated powdered sugar   it is a tiny mouthful heaven   too chewy  very flavorful   highly recommend yummy treat   you familiar story c s  lewis   the lion  witch  the wardrobe    this is the treat seduces edmund selling his brother sisters the witch 
Actual Summary :   delight  says it all
Predicted Summary :  great flavor

T